In [466]:
import re, sys, copy, numpy as np, copy
from math import sqrt, lcm
from aoclib import list2string
from collections import defaultdict
from itertools import combinations
from operator import add, mul

In [467]:
global d1global
d1global = []

def get_eff_pow(group):
    return group['n'] * group['dmg'][0]

def get_dmg_multiplier(group, dmg_type):
    if dmg_type in group['immunities']:
        return 0
    if dmg_type in group['weaknesses']:
        return 2
    return 1

def Fight(is_, inf_, boost = 0):    
    ## Target selection phase
    
    # Immune system
    is_att_inf_def_pairs = []
    avail_def = list(inf_.keys())
    for i in sorted(is_, key = lambda x: (- is_[x]['n'] * is_[x]['dmg'][0], - is_[x]['init'])):
        att_group = is_[i]

        if avail_def:
            target = sorted(avail_def, \
                        key = lambda x: (- get_dmg_multiplier(inf_[x], att_group['dmg'][1]), \
                                         - get_eff_pow(inf_[x]), \
                                         - inf_[x]['init']))[0]
            if get_dmg_multiplier(inf_[target], att_group['dmg'][1]) == 0:
                continue
            avail_def.remove(target)            
            is_att_inf_def_pairs.append((i, target))
            
    # Infection
    inf_att_is_def_pairs = []
    avail_def = list(is_.keys())
    for i in sorted(inf_, key = lambda x: (- inf_[x]['n'] * inf_[x]['dmg'][0], - inf_[x]['init'])):
        att_group = inf_[i]

        if avail_def:
            target = sorted(avail_def, \
                        key = lambda x: (- get_dmg_multiplier(is_[x], att_group['dmg'][1]), \
                                         - get_eff_pow(is_[x]), \
                                         - is_[x]['init']))[0]
            if get_dmg_multiplier(is_[target], att_group['dmg'][1]) == 0:
                continue
            avail_def.remove(target)
            inf_att_is_def_pairs.append((i, target))

    #print(inf_)
    is_att_inf_def_pairs.sort(key = lambda x: is_[x[0]]['init'], reverse = True)
    inf_att_is_def_pairs.sort(key = lambda x: inf_[x[0]]['init'], reverse = True)
    #print(inf_att_is_def_pairs[0][0])
    #print(inf_[inf_att_is_def_pairs[0][0]])
    #sys.exit()

    ## Attacking phase
    damage_dealt = False
    while inf_att_is_def_pairs or is_att_inf_def_pairs:
        if inf_att_is_def_pairs and \
            (not is_att_inf_def_pairs or \
             inf_[inf_att_is_def_pairs[0][0]]['init'] > is_[is_att_inf_def_pairs[0][0]]['init']):
                pair = inf_att_is_def_pairs.pop(0)
                kills = get_eff_pow(inf_[pair[0]])*get_dmg_multiplier(is_[pair[1]], inf_[pair[0]]['dmg'][1]) // is_[pair[1]]['hp']
                sav = is_[pair[1]]['n']
                is_[pair[1]]['n'] -= kills
                is_[pair[1]]['n'] = max(is_[pair[1]]['n'], 0)
                if sav != is_[pair[1]]['n']:
                    damage_dealt = True
                #print("Infection", pair[0] + 1, "attacking immune system", pair[1] + 1, "- Kills:", kills)
                #d1global.append(("attacker", copy.deepcopy(inf_[pair[0]]), "team", 0, "attacked idx", pair[1], "kills", kills, list(is_[pair[1]].values()), len(is_)))
                #print(pair[0], inf_[pair[0]])
        else:
                pair = is_att_inf_def_pairs.pop(0)
                dmg = get_eff_pow(is_[pair[0]])
                multiplier = get_dmg_multiplier(inf_[pair[1]], is_[pair[0]]['dmg'][1])
                dmg *= multiplier
                kills = dmg // inf_[pair[1]]['hp']
                sav = inf_[pair[1]]['n']
                inf_[pair[1]]['n'] -= kills
                inf_[pair[1]]['n'] = max(inf_[pair[1]]['n'], 0)
                if sav != inf_[pair[1]]['n']:
                    damage_dealt = True
                if pair[1] == 3:
                    d1global.append(("attacker hp", is_[pair[0]]['hp'], "eff power", get_eff_pow(is_[pair[0]]), dmg, "multiplier", multiplier, "kills", kills, inf_[pair[1]]['n']))
                #print("Immune system", pair[0] + 1, "attacking infection", pair[1] + 1, "- Kills:", kills)
                #d1global.append(("attacker", copy.deepcopy(is_[pair[0]]), "team", 1, "attacked idx", pair[1], "kills", kills, list(inf_[pair[1]].values()), len(inf_)))

    #print(inf_)
    
    ## Deletion phase
    is__ = is_.copy()
    [is__.pop(x) for x in is_ if is_[x]['n'] == 0]
    inf__ = inf_.copy()
    [inf__.pop(x) for x in inf_ if inf_[x]['n'] == 0]
        
    return is__, inf__, damage_dealt

In [468]:
# Parsing input
test = False
if not test:
    input_file = open('24.txt').read().strip()
    input_is, input_inf = input_file.split("\n\n")
    input_is, input_inf = input_is.splitlines()[1:], input_inf.splitlines()[1:]
else:
    input_file = open('24ex.txt').read().strip()
    input_is, input_inf = input_file.split("\n\n")
    input_is, input_inf = input_is.splitlines()[1:], input_inf.splitlines()[1:]
    for k in range(0, len(input_is) // 2):
        input_is[k] += input_is[k + 1]
        input_is.pop(k + 1)
    for k in range(0, len(input_inf) // 2):
        input_inf[k] += input_inf[k + 1]
        input_inf.pop(k + 1)

regexp_num = r'(\d+) [\w ]* (\d+) [\w ,;()]* (\d+) (\w+) [\w ,()]* (\d+)'

immune_system = dict()
for k, input_group in enumerate(input_is):
    n, hp, dmg, dmg_type, initiative = re.findall(regexp_num, input_group)[0]
    immunities = re.findall(r'immune to ([, \w]+)[\);]', input_group)
    if immunities:
        immunities = immunities[0].split(", ")
    weaknesses = re.findall(r'weak to ([, \w]+)[\);]', input_group)
    if weaknesses:
        weaknesses = weaknesses[0].split(", ")
    immune_system[k] = {'n': int(n), 'hp': int(hp), 'dmg': [int(dmg), dmg_type], 'init': int(initiative), \
                        'immunities': immunities, 'weaknesses': weaknesses}

infection = dict()
for k, input_group in enumerate(input_inf):
    n, hp, dmg, dmg_type, initiative = re.findall(regexp_num, input_group)[0]
    immunities = re.findall(r'immune to ([, \w]+)[\);]', input_group)
    if immunities:
        immunities = immunities[0].split(", ")
    weaknesses = re.findall(r'weak to ([, \w]+)[\);]', input_group)
    if weaknesses:
        weaknesses = weaknesses[0].split(", ")
    infection[k] = {'n': int(n), 'hp': int(hp), 'dmg': [int(dmg), dmg_type], 'init': int(initiative), \
                        'immunities': immunities, 'weaknesses': weaknesses} 

In [469]:
### Part 1
# Simulating
is_, inf_ = copy.deepcopy(immune_system), copy.deepcopy(infection)

while is_ and inf_:
    #print("Fight", it := it + 1)
    #print("")
    is_, inf_, damage_dealt = Fight(is_, inf_, boost = 0)
    #print(is_, inf_)

if is_:
    print("Winning army: Immune system; Units:", sum(x['n'] for x in is_.values()))
else:
    print("Winning army: Infection; Units:", sum(x['n'] for x in inf_.values()))
   

Winning army: Infection; Units: 16530


In [470]:
# Part 2
boost = 0
while True:
    boost += 1
    is_, inf_ = copy.deepcopy(immune_system), copy.deepcopy(infection)
    # Applying boost here
    for i in is_:
        is_[i]['dmg'][0] += boost

    tie = False
    it = 0
    while is_ and inf_:
        it += 1
        is_, inf_, damage_dealt = Fight(is_, inf_, boost = 0)
        if boost == 40:
            if it % 10000 == 0:
                print([(is_[x]['hp'], is_[x]['n']) for x in is_])
                print([inf_[x]['n'] for x in inf_])
        if not damage_dealt:
            print("Boost", boost, "Tie.")
            print("No damage dealt in fight no.", it)
            tie = True
            break
    if tie:
        continue
    if is_:
        print("Boost", boost, "Winning army: Immune system; Units:", sum(x['n'] for x in is_.values()))
        break
    elif inf_:
        print("Boost", boost, "Winning army: Infection; Units:", sum(x['n'] for x in inf_.values()))
        

Boost 1 Winning army: Infection; Units: 16445
Boost 2 Winning army: Infection; Units: 16282
Boost 3 Winning army: Infection; Units: 16159
Boost 4 Winning army: Infection; Units: 16050
Boost 5 Winning army: Infection; Units: 15883
Boost 6 Winning army: Infection; Units: 15768
Boost 7 Winning army: Infection; Units: 15667
Boost 8 Winning army: Infection; Units: 15522
Boost 9 Winning army: Infection; Units: 15332
Boost 10 Winning army: Infection; Units: 15073
Boost 11 Winning army: Infection; Units: 14895
Boost 12 Winning army: Infection; Units: 14778
Boost 13 Winning army: Infection; Units: 14664
Boost 14 Winning army: Infection; Units: 14556
Boost 15 Winning army: Infection; Units: 14301
Boost 16 Winning army: Infection; Units: 14167
Boost 17 Winning army: Infection; Units: 14005
Boost 18 Winning army: Infection; Units: 13666
Boost 19 Winning army: Infection; Units: 13282
Boost 20 Winning army: Infection; Units: 13045
Boost 21 Winning army: Infection; Units: 13113
Boost 22 Winning army: